In [1]:
import pandas

In [2]:
import random

In [3]:
from sklearn.neighbors import KNeighborsClassifier

In [4]:
abalones = pandas.read_excel("dataset_processado.xlsx");
test = pandas.read_excel("app_processado.xlsx");

accuracy = {}; #Grupo de parâmetros: acurácia

In [5]:
#Parâmetros de teste
numReps = 10;
numChanges = 1;
randomSplit = 0.3;


In [6]:
#Geração Aleatória
for n in range(numChanges):
    weights = [random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(),random.random(), random.randrange(3, 50, 2)]    
    accuracyResults = [];

    for n in range(numReps):
        knn = KNeighborsClassifier(n_neighbors = weights[8]);

        #Selecionando índices para teste e treino
        trainingIndexes = [];
        testIndexes = [];
        n = 0;
        for item in abalones["sex"]:
            if random.random() > randomSplit:
                trainingIndexes.append(n)
            else:
                testIndexes.append(n)
            n+=1

        #Criando os dataframes de treino e teste e separando os classificadores
        testData = abalones.drop(trainingIndexes);
        testClasses = testData["type"];
        testData = testData.drop(columns=["type"]);

        trainingData = abalones.drop(testIndexes);
        classifications = trainingData["type"]
        trainingData = trainingData.drop(columns=["type"])

        #Aplicando pesos
        n = 0;
        for col in trainingData.columns:
            trainingData[col] = trainingData[col]*weights[n]
            testData[col] = testData[col]*weights[n]
            n+=1

        #Aplicando K-NN
        knn.fit(trainingData, classifications)

        results = knn.predict(testData);

        #Testando acurácia
        n = 0;
        correctPredictions = 0;
        for item in testClasses:
            if item == results[n]:
                correctPredictions += 1;
            n+=1  

        acc = correctPredictions/len(results)
        accuracyResults.append(acc)

    meanAcc = sum(accuracyResults)/len(accuracyResults)
    print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
    print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
          + "\nSexo: " + str(weights[0]) 
          + "\nComprimento: " + str(weights[1])
          + "\nDiâmetro: " + str(weights[2])
          + "\nAltura: " + str(weights[3])
          + "\nPeso cheio: " + str(weights[4])
          + "\nPeso vazio: " + str(weights[5])
          + "\nPeso das vísceras: " + str(weights[6])
          + "\nPeso da concha: " + str(weights[7])
         )
    accuracy[str(weights)] = meanAcc

Acurácia média em 10 tentativas: 0.615181100280551

----- PARÂMETROS -----
Número de Vizinhos: 15
Sexo: 0.9118833983335068
Comprimento: 0.4557947057445374
Diâmetro: 0.6221925600352841
Altura: 0.6685002769060638
Peso cheio: 0.6669205678059897
Peso vazio: 0.1632553921233043
Peso das vísceras: 0.44186257614636604
Peso da concha: 0.3451131350010799


In [7]:
#Forward selection

def forwardSelection(baseWeights, baseAcc):
    m = 0;
    accs = [];
    for col in abalones.columns[:8]:
        weights = baseWeights.copy();
        weights[m] = 1
        accuracyResults = [];
    
        for num in range(numReps):
            knn = KNeighborsClassifier(n_neighbors = weights[8]);

            #Selecionando índices para teste e treino
            trainingIndexes = [];
            testIndexes = [];
            n = 0;
            for item in abalones["sex"]:
                if random.random() > randomSplit:
                    trainingIndexes.append(n)
                else:
                    testIndexes.append(n)
                n+=1

            #Criando os dataframes de treino e teste e separando os classificadores
            testData = abalones.drop(trainingIndexes);
            testClasses = testData["type"];
            testData = testData.drop(columns=["type"]);

            trainingData = abalones.drop(testIndexes);
            classifications = trainingData["type"]
            trainingData = trainingData.drop(columns=["type"])

            #Aplicando pesos
            n = 0;
            for col in trainingData.columns:
                trainingData[col] = trainingData[col]*weights[n]
                testData[col] = testData[col]*weights[n]
                n+=1

            #Aplicando K-NN
            knn.fit(trainingData, classifications)

            results = knn.predict(testData);

            #Testando acurácia
            n = 0;
            correctPredictions = 0;
            for item in testClasses:
                if item == results[n]:
                    correctPredictions += 1;
                n+=1  

            acc = correctPredictions/len(results)
            accuracyResults.append(acc)

        meanAcc = sum(accuracyResults)/len(accuracyResults)
        '''print("Acurácia média em " + str(numReps) + " tentativas: " + str(meanAcc))
        print("\n----- PARÂMETROS -----\nNúmero de Vizinhos: " + str(weights[8])
              + "\nSexo: " + str(weights[0]) 
              + "\nComprimento: " + str(weights[1])
              + "\nDiâmetro: " + str(weights[2])
              + "\nAltura: " + str(weights[3])
              + "\nPeso cheio: " + str(weights[4])
              + "\nPeso vazio: " + str(weights[5])
              + "\nPeso das vísceras: " + str(weights[6])
              + "\nPeso da concha: " + str(weights[7]) + "\n\n"
             )'''
        accs.append(meanAcc)
        m+=1
    
    #print("Acurácias finais: " + str(accs) + '\n\n')
    
    #Índice do melhor resultado
    ind = accs.index(max(accs))
    
    #Conferindo se houve melhora nessa recursão, caso não, já retorna a última lista de parâmetros
    if accs[ind] <= baseAcc:
        print("Resultado final: " + str(accs[ind]) + "%")
        return baseWeights
    #Caso haja melhora, a função será chamada novamente, com uma nova base de pesos e com a melhor acurácia
    else:
        newWeights = baseWeights.copy();
        newWeights[ind] = 1;
        return forwardSelection(newWeights, accs[ind])
    
    
for neighs in [5,15,25,50]:
    print(forwardSelection([0,0,0,0,0,0,0,0,neighs], 0))

#bestSet = forwardSelection([0,0,0,0,0,0,0,0,5], 0)

Resultado final: 0.6197105641382942%
[1, 0, 0, 0, 0, 1, 0, 1, 5]
Resultado final: 0.6381094519383159%
[0, 0, 0, 0, 1, 1, 0, 1, 15]
Resultado final: 0.6444495238904868%
[0, 1, 1, 1, 0, 1, 0, 1, 25]
Resultado final: 0.653018967907297%
[0, 0, 0, 1, 1, 1, 0, 1, 50]
